In [31]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from torchvision import transforms, datasets, models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

In [32]:
import os

data_dir = 'C:\\Users\\cbz\\Downloads\\train'

if os.path.exists(data_dir):
    contents = os.listdir(data_dir)
    print(contents)
else:
    print(f"The folder '{data_dir}' does not exist.")


['Closed', 'no_yawn', 'Open', 'yawn']


In [33]:
# Define data transforms for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [34]:
# Load the data using PyTorch's ImageFolder
#image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
#dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64, shuffle=True) for x in ['train', 'val']}

import os

data_dir = 'C:\\Users\\cbz\\Downloads\\train'

# try:
#     image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
#     dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64, shuffle=True) for x in ['train', 'val']}
# except Exception as e:
#     print(f"An error occurred: {str(e)}")

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\cbz\\Downloads\\train\\train'

In [ ]:
# Define the model
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(image_datasets['train'].classes))

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\cbz/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


NameError: name 'image_datasets' is not defined

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training function
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
# Train the model
model = train_model(model, criterion, optimizer, num_epochs=10)

In [ ]:
# Save the trained model
torch.save(model.state_dict(), "drowsiness_resnet50.pth")

In [ ]:
# Evaluate the model
def evaluate_model(model, dataloader):
    model.eval()
    running_corrects = 0

    for inputs, labels in dataloader:
        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)

    accuracy = running_corrects.double() / len(dataloader.dataset)
    return accuracy.item()


In [ ]:
# Evaluate on validation data
val_accuracy = evaluate_model(model, dataloaders['val'])
print('Validation Accuracy: {:.4f}'.format(val_accuracy))

In [ ]:
# Generate predictions
def predict(model, dataloader):
    model.eval()
    all_preds = []

    for inputs, _ in dataloader:
        with torch.no_grad():
            outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())

    return all_preds

In [ ]:
# Predict on test data
y_true = image_datasets['val'].targets
y_pred = predict(model, dataloaders['val'])

In [ ]:
# Confusion matrix
confusion_mtx = confusion_matrix(y_true, y_pred)

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# Print classification report
class_report = classification_report(y_true, y_pred, target_names=image_datasets['train'].classes)
print('Classification Report:\n', class_report)